# Árboles de decisión

Un árbol de decisión es un tipo de modelo de conocimiento que puede
generarse a partir de datos y que puede usarse tanto para clasificación como regresión.

En este último caso, se denomina árbol de regresión, aunque su construcción es análoga a la del árbol de decisión.

>Note: La idea conceptual de los árboles de decisión es dividir el conjunto de datos de forma jerárquica en trozos cada vez menores hasta llegar a trozos donde los ejemplos son todos, o prácticamente todos, de la misma clase. La forma de dividir los datos en trozos se hace a partir de los valores de los atributos.

En concreto, cada vez que se divide un conjunto de datos en dos o más trozos (y, así, de forma recursiva), suele ser un único atributo el que se usa. Dependiendo del valor del atributo, se aplican filtros a los datos para generar trozos de datos menores cuyas filas han sido filtradas para que tengan únicamente ciertos valores del atributo en cuestión.

Para construir un árbol de decisión, el algoritmo de construcción debe asignar a cada nodo del árbol el atributo más apropiado en cada caso. El atributo más apropiado debería ser aquel que, mediante sus valores posibles, mejor distingue las instancias de una clase o varias clases con respecto a las demás, esto es, el atributo que mejor discrimine las clases de los ejemplos.

Existen diferentes algoritmos de árboles de decisión, particularmente estudiaremos `CART` (classification and regression trees en inglés). Como su propio nombre indica, permite realizar tanto clasificación como regresión usando árboles como modelo. A diferencia de otros modelos `CART` produce únicamente árboles binarios; esto
es, desde cada nodo parten a lo sumo dos aristas. La librería `scikit-learn` implementa una versión optimizada del algoritmo `CART`, pero no admite atributos categóricos, tan solo numéricos.

## Ejemplo

En el Programa, se muestra un ejemplo de aplicación del algoritmo CART de la librería `scikit-learn` para la construcción de árboles de decisión mediante validación cruzada con 10 bolsas. El algoritmo se ha aplicado a tres conjuntos de datos distintos: `iris`, `wine` y `breast-cancer`.



In [ ]:
from sklearn import tree
from sklearn import datasets
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn import preprocessing
from evaluacion_funciones import *
import warnings
warnings.filterwarnings("ignore")

In [ ]:
seed=42

In [ ]:
# Cargamos el conjunto de datos
datos_iris = datasets.load_iris()
datos_wine = datasets.load_wine()
datos_breast = datasets.load_breast_cancer()

X = {
    'IRIS': datos_iris.data,
    'WINE': datos_wine.data,
    'BREAST': datos_breast.data
}

y = {
    'IRIS': datos_iris.target,
    'WINE': datos_wine.target,
    'BREAST': datos_breast.target
}



In [ ]:
# 5) Construcción del algoritmo de aprendizaje.
algoritmos = {'DT': tree.DecisionTreeClassifier(criterion='gini', random_state=seed)}

In [ ]:
# Métricas de evaluación.
metricas = {
  'ACC':    metrics.accuracy_score,
  'PREC':   lambda y_true, y_pred:
            metrics.precision_score(y_true, y_pred, average='micro'),
  'RECALL': lambda y_true, y_pred:
            metrics.recall_score(y_true, y_pred, average='micro'),
  'F1':     lambda y_true, y_pred:
            metrics.f1_score(y_true, y_pred, average='micro')
}

In [ ]:
# 1) Partición externa
# 2) Extracción de características (en caso de trabajar con imágenes)

# 3) Estandarizacion de los datos
standardizer = preprocessing.StandardScaler()
for nombre, exp in X.items():
    X[nombre] = standardizer.fit_transform(X[nombre])

# 4) Selección de atributos

In [ ]:
# 5.1) Validación cruzada interna y Optimización de los hiperparámetros
y_pred = {}
for nombre, exp in X.items():
    y_pred[nombre] = cross_val_predict(algoritmos['DT'], X[nombre], y[nombre], 
                                       cv=KFold(n_splits=5, shuffle=True, random_state=seed))

In [ ]:
# Mostramos el resultado de varias métricas
results={}
for nombre, exp in X.items():
    results[nombre] = evaluacion(y[nombre], y_pred[nombre], metricas)
    print("Matriz de confusión (%s):\n%s" % (nombre, metrics.confusion_matrix(y[nombre], y_pred[nombre])))
    print("Tabla de métricas (%s):\n%s" % (nombre, metrics.classification_report(y[nombre], y_pred[nombre], digits=3)))

In [ ]:
# 5.2) Entrenamiento del modelo definitivo (usamos 2 atributos para poder obtener gráficos en 2D)
for nombre, exp in X.items():
    modelo_definitivo = algoritmos['DT'].fit(X[nombre][:,:2], y[nombre])
    mapa_modelo_clasif_2d(X[nombre][:,:2], y[nombre], modelo_definitivo, results[nombre], nombre)

In [ ]:
# 5.3) Ploteamos el árbol con todos los datos para visualización árbol.
for nombre, exp in X.items():
    modelo_completo = algoritmos['DT'].fit(X[nombre], y[nombre])
    plt.figure()
    tree.plot_tree(modelo_completo,filled=True)  
    plt.savefig('tree_' + nombre + '.pdf',format='pdf',bbox_inches = "tight")

En concreto, en el Programa se han definido tres experimentos, en los que se aplica el algoritmo `CART` (implementado en la clase DecisionTreeClassifier del paquete tree de la librería `scikit-learn`) a los tres conjuntos de datos antes mencionados: `iris`, `wine` y `breast-cancer`. Una vez definidos los experimentos, en el paso 5 del Programa  se realiza la validación cruzada mediante la función `cross_val_predict()` de
`scikit-learn`, tal como hemos procedido en anteriores programas.

Los resultados producidos por el Programa son los siguientes:

- Mapa de regiones de clasificación del modelo generado en el experimento con el conjunto de datos “iris”.

- Mapa de regiones de clasificación del modelo generado en el experimento con el conjunto de datos “wine”.

- Mapa de regiones de clasificación del modelo generado en el experimento con el conjunto de datos “breast-cancer”.

- Árbol de decisión generado por CART para el conjunto “iris”.

- Árbol de decisión generado por CART para el conjunto “wine”.

- Árbol de decisión generado por CART para el conjunto “breast-cancer”.

- Matrices de confusión y tablas de métricas de los 3 experimentos.